In [1]:
# import the i8080 module
import _i8080 as i8080

dut = i8080.i8080_new()

i8080_exec
Initialising i8080 object


In [2]:
# INR Increment Register or Memory Tests

inr_opcode = 0b0000_0100

registers = {
    "b": 0b000,
    "c": 0b001,
    "d": 0b010,
    "e": 0b011,
    "h": 0b100,
    "l": 0b101,
    "m": 0b110,
    "a": 0b111,
}

for register, register_code in registers.items():
    print(f"Testing INR {register}")

    for i in range(0,256):
        dut.reset()

        if register == "m":
            # Set the memory address to 0x1000
            # otherwise the memory address will be 0x0000, where the instruction is stored
            # dut.set_reg("h", 0x10)
            dut.h = 0x10
        
        #dut.set_reg(register, i)
        setattr(dut, register, i)

        opcode = inr_opcode | (register_code << 3)
        dut.set_memory(0, opcode)
        dut.run_instruction()

        expected = (i + 1) & 0xFF
        #actual = dut.get_reg(register)
        actual = getattr(dut, register)

        if expected != actual:
            print(f"{register} failed at {i} expected {expected} got {actual}")
            break

Testing INR b
Testing INR c
Testing INR d
Testing INR e
Testing INR h
Testing INR l
Testing INR m
Testing INR a


In [3]:
# DCR Decrement Register or Memory Tests

dcr_opcode = 0b0000_0101

registers = {
    "b": 0b000,
    "c": 0b001,
    "d": 0b010,
    "e": 0b011,
    "h": 0b100,
    "l": 0b101,
    "m": 0b110,
    "a": 0b111,
}

for register, register_code in registers.items():
    print(f"Testing DCR {register}")

    for i in range(0,256):
        dut.reset()

        if register == "m":
            # Set the memory address to 0x1000
            # otherwise the memory address will be 0x0000, where the instruction is stored
            #dut.set_reg("h", 0x10)
            dut.h = 0x10
        #dut.set_reg(register, i)
        setattr(dut, register, i)

        opcode = dcr_opcode | (register_code << 3)
        dut.set_memory(0, opcode)
        dut.run_instruction()

        expected = (i - 1) & 0xFF
        #actual = dut.get_reg(register)
        actual = getattr(dut, register)

        if expected != actual:
            print(f"{register} failed at {i} expected {expected} got {actual}")
            break

Testing DCR b
Testing DCR c
Testing DCR d
Testing DCR e
Testing DCR h
Testing DCR l
Testing DCR m
Testing DCR a


In [4]:
# CMA Complement Accumulator Tests

cma_opcode = 0b0010_1111

for i in range(0,256):
    dut.reset()

    #dut.set_reg("a", i)
    dut.a = i

    dut.set_memory(0, cma_opcode)
    dut.run_instruction()

    expected = ~i & 0xFF
    #actual = dut.get_reg("a")
    actual = dut.a

    if expected != actual:
        print(f"CMA failed at {i} expected {expected} got {actual}")
        break

In [5]:
# DAA Decimal Adjust Accumulator Tests

daa_opcode = 0b0010_0111

for i in range(0,256):
    dut.reset()

    dut.a = i

    dut.set_memory(0, daa_opcode)
    dut.run_instruction()

    actual = dut.a

    high_nibble = (actual >> 4) & 0xF
    low_nibble = actual & 0xF


    high_nibble_i = (i >> 4) & 0xF
    low_nibble_i = i & 0xF

    if low_nibble_i > 9 or dut.ac:
        low_nibble_i += 6
        low_nibble_i &= 0xF

    if high_nibble_i > 9 or dut.cy:
        high_nibble += 6
        high_nibble &= 0xF

    if high_nibble_i != high_nibble or low_nibble_i != low_nibble:
        print(f"DAA failed at {i} expected {high_nibble}{low_nibble} got {high_nibble_i}{low_nibble_i}")
        break

DAA failed at 10 expected 10 got 00


In [6]:
# Test Call and Return

dut.reset()

call_opcode = 0b1100_1101
ret_opcode = 0b1100_1001

dut.sp = 0x1000

dut.set_memory(0, call_opcode)
dut.set_memory(1, 0x03)
dut.set_memory(2, 0x03)

dut.set_memory(0x0303, ret_opcode)

dut.run_instruction()
dut.run_instruction()

# Program Counter should be 0x0003
if dut.pc != 0x0003:
    print(f"Call and Return failed, expected PC = 0x0003, got {dut.pc}")


In [7]:
# Check LXI

lxi_opcode = 0b0000_0001

registers = {
    "bc": 0b00,
    "de": 0b01,
    "hl": 0b10,
    "sp": 0b11,
}

for register, register_code in registers.items():
    print(f"Testing LXI {register}")

    for i in range(128,512):
        dut.reset()

        opcode = lxi_opcode | (register_code << 4)
        dut.set_memory(0, opcode)

        dut.set_memory(1, i & 0xFF)
        dut.set_memory(2, (i >> 8) & 0xFF)
        dut.run_instruction()

        expected = i

        actual = getattr(dut, register)

        if expected != actual:
            print(f"{register} failed at {i} expected {expected} got {actual}")
            break

Testing LXI bc
Testing LXI de
Testing LXI hl
Testing LXI sp
